<a href="https://colab.research.google.com/github/AlkinD/Project_Collection/blob/main/TranstoIndiclang.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Transliteration of indic languages**

**From Uploading File from Kaggle**

In [ ]:
!nvidia-smi

In [ ]:
!pip install -q kaggle

In [ ]:
from google.colab import files
files.upload()

In [ ]:
#create a kaggle folder
#!rm -r ~/..kaggle
#!mkdir ~/..kaggle

!mkdir /root/.kaggle
!mkdir /root/.kaggle/input
!mkdir /root/.kaggle/input/hindi-to-hinglish
#!rm -r /content/kaggle

In [ ]:
!ls

In [ ]:
#copy to the kaggle.json to the folder created
!cp kaggle.json /root/.kaggle/

In [ ]:
#permission for the jason to act
!chmod 600 /root/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d parthplc/hindi-to-hinglish

 59% 9.00M/15.2M [00:00<00:00, 82.6MB/s]
100% 15.2M/15.2M [00:00<00:00, 97.7MB/s]


In [ ]:
!ls

hindi-to-hinglish.zip  kaggle.json  sample_data


In [ ]:
#unzipping the zip files and deleting the zip files
!unzip hindi-to-hinglish.zip -d /root/.kaggle/input/hindi-to-hinglish
!rm hindi-to-hinglish.zip
#!unzip \*.zip  && rm *.zip

Archive:  hindi-to-hinglish.zip
  inflating: /root/.kaggle/input/hindi-to-hinglish/train1.csv  
  inflating: /root/.kaggle/input/hindi-to-hinglish/train2.csv  
  inflating: /root/.kaggle/input/hindi-to-hinglish/valid1.csv  
  inflating: /root/.kaggle/input/hindi-to-hinglish/valid2.csv  


In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os

In [ ]:
#train = pd.read_csv("/root/.kaggle/input/hindi-to-hinglish/train1.csv")


In [ ]:
#train.shape

(61824, 3)

In [ ]:
# !pip install transformers
!pip install pytorch_lightning==0.8.1

     |████████████████████████████████| 293 kB 5.3 MB/s 
     |████████████████████████████████| 829 kB 35.4 MB/s 
     |████████████████████████████████| 596 kB 42.5 MB/s 
  Created wheel for future: filename=future-0.18.2-py3-none-any.whl size=491070 sha256=4b64b6ee00dce6667186b0bd6073e9d73467f45e8efd3254f74698f7bc3c5f7e
  Stored in directory: /root/.cache/pip/wheels/56/b0/fe/4410d17b32f1f0c3cf54cdfb2bc04d7b4b8f4ae377e2229ba0
Successfully built future
  Attempting uninstall: PyYAML
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
  Attempting uninstall: future
    Found existing installation: future 0.16.0
    Uninstalling future-0.16.0:
      Successfully uninstalled future-0.16.0


In [ ]:
!pip install transformers==4.11.3.
!pip install sentencepiece

     |████████████████████████████████| 2.9 MB 5.6 MB/s 
     |████████████████████████████████| 3.3 MB 31.1 MB/s 
     |████████████████████████████████| 895 kB 24.2 MB/s 
     |████████████████████████████████| 61 kB 463 kB/s 
     |████████████████████████████████| 1.2 MB 5.6 MB/s 


In [ ]:
import argparse
import glob
import os
import json
import time
import logging
import random
import re
from itertools import chain
from string import punctuation

import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize

import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
import pytorch_lightning as pl

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
#train.head()

In [ ]:
pl.__version__

In [ ]:
from transformers import (
    AdamW,
    MT5ForConditionalGeneration,
    AutoTokenizer,
    get_linear_schedule_with_warmup
)

In [ ]:
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

set_seed(42)

In [ ]:
class T5FineTuner(pl.LightningModule):
    def __init__(self, hparams):
        super(T5FineTuner, self).__init__()
        self.hparams = hparams

        self.model = MT5ForConditionalGeneration.from_pretrained(hparams.model_name_or_path)
        self.tokenizer = AutoTokenizer.from_pretrained(hparams.tokenizer_name_or_path)

    def is_logger(self):
        return True

    def forward(
            self, input_ids, attention_mask=None, decoder_input_ids=None, decoder_attention_mask=None, labels=None
    ):
        return self.model(
            input_ids,
            attention_mask=attention_mask,
            decoder_input_ids=decoder_input_ids,
            decoder_attention_mask=decoder_attention_mask,
            labels=labels,
        )

    def _step(self, batch):
        labels = batch["target_ids"]
        labels[labels[:, :] == self.tokenizer.pad_token_id] = -100

        outputs = self(
            input_ids=batch["source_ids"],
            attention_mask=batch["source_mask"],
            labels=labels,
            decoder_attention_mask=batch['target_mask']
        )

        loss = outputs[0]

        return loss

    def training_step(self, batch, batch_idx):
        loss = self._step(batch)

        tensorboard_logs = {"train_loss": loss}
        return {"loss": loss, "log": tensorboard_logs}

    def training_epoch_end(self, outputs):
        avg_train_loss = torch.stack([x["loss"] for x in outputs]).mean()
        tensorboard_logs = {"avg_train_loss": avg_train_loss}
        return {"avg_train_loss": avg_train_loss, "log": tensorboard_logs, 'progress_bar': tensorboard_logs}

    def validation_step(self, batch, batch_idx):
        loss = self._step(batch)
        return {"val_loss": loss}

    def validation_epoch_end(self, outputs):
        avg_loss = torch.stack([x["val_loss"] for x in outputs]).mean()
        tensorboard_logs = {"val_loss": avg_loss}
        return {"avg_val_loss": avg_loss, "log": tensorboard_logs, 'progress_bar': tensorboard_logs}

    def configure_optimizers(self):
        "Prepare optimizer and schedule (linear warmup and decay)"

        model = self.model
        no_decay = ["bias", "LayerNorm.weight"]
        optimizer_grouped_parameters = [
            {
                "params": [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
                "weight_decay": self.hparams.weight_decay,
            },
            {
                "params": [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)],
                "weight_decay": 0.0,
            },
        ]
        optimizer = AdamW(optimizer_grouped_parameters, lr=self.hparams.learning_rate, eps=self.hparams.adam_epsilon)
        self.opt = optimizer
        return [optimizer]

    def optimizer_step(self, epoch, batch_idx, optimizer, optimizer_idx, second_order_closure=None):
        if self.trainer.use_tpu:
            xm.optimizer_step(optimizer)
        else:
            optimizer.step()
        optimizer.zero_grad()
        self.lr_scheduler.step()

    def get_tqdm_dict(self):
        tqdm_dict = {"loss": "{:.3f}".format(self.trainer.avg_loss), "lr": self.lr_scheduler.get_last_lr()[-1]}

        return tqdm_dict

    def train_dataloader(self):
        train_dataset = get_dataset(tokenizer=self.tokenizer, type_path="train1", args=self.hparams)
        dataloader = DataLoader(train_dataset, batch_size=self.hparams.train_batch_size, drop_last=True, shuffle=True,
                                num_workers=4)
        t_total = (
                (len(dataloader.dataset) // (self.hparams.train_batch_size * max(1, self.hparams.n_gpu)))
                // self.hparams.gradient_accumulation_steps
                * float(self.hparams.num_train_epochs)
        )
        scheduler = get_linear_schedule_with_warmup(
            self.opt, num_warmup_steps=self.hparams.warmup_steps, num_training_steps=t_total
        )
        self.lr_scheduler = scheduler
        return dataloader

    def val_dataloader(self):
        val_dataset = get_dataset(tokenizer=self.tokenizer, type_path="valid1", args=self.hparams)
        return DataLoader(val_dataset, batch_size=self.hparams.eval_batch_size, num_workers=4)

In [ ]:
logger = logging.getLogger(__name__)

class LoggingCallback(pl.Callback):
        def on_validation_end(self, trainer, pl_module):
            logger.info("***** Validation results *****")
            if pl_module.is_logger():
                  metrics = trainer.callback_metrics
                  # Log results
                  for key in sorted(metrics):
                    if key not in ["log", "progress_bar"]:
                      logger.info("{} = {}\n".format(key, str(metrics[key])))

        def on_test_end(self, trainer, pl_module):
            logger.info("***** Test results *****")

            if pl_module.is_logger():
                metrics = trainer.callback_metrics

                  # Log and save results to file
                output_test_results_file = os.path.join(pl_module.hparams.output_dir, "test_results.txt")
                with open(output_test_results_file, "w") as writer:
                    for key in sorted(metrics):
                          if key not in ["log", "progress_bar"]:
                            logger.info("{} = {}\n".format(key, str(metrics[key])))
                            writer.write("{} = {}\n".format(key, str(metrics[key])))


Let's define the hyperparameters and other arguments. You can overide this dict for specific task as needed. While in most of cases you'll only need to change the data_dirand output_dir.

Here the batch size is 8 and gradient_accumulation_steps are 8 so the effective batch size is 64

In [ ]:
args_dict = dict(
    data_dir="", # path for data files
    output_dir="", # path to save the checkpoints
    model_name_or_path='google/mt5-base',
    tokenizer_name_or_path='google/mt5-base',
    max_seq_length=384,   #512,
    learning_rate=3e-4,
    weight_decay=0.0,
    adam_epsilon=1e-8,
    warmup_steps=0,
    train_batch_size=2,
    eval_batch_size=4,
    num_train_epochs=2,
    gradient_accumulation_steps=8,
    n_gpu=1,
    early_stop_callback=False,
    fp_16=False, # if you want to enable 16-bit training then install apex and set this to true
    opt_level='O1', # you can find out more on optimisation levels here https://nvidia.github.io/apex/amp.html#opt-levels-and-properties
    max_grad_norm=1.0, # if you enable 16-bit training then set this to a sensible value, 0.5 is a good default
    seed=42,
)
!ls

kaggle.json  sample_data


In [ ]:
train_path = "/root/.kaggle/input/hindi-to-hinglish/train1.csv"
val_path = "/root/.kaggle/input/hindi-to-hinglish/valid1.csv"

"""train = pd.read_csv(train_path)
print (train.head())"""

# tokenizer = AutoTokenizer.from_pretrained('google/mt5-small')

                                               input  ...     lang
0          beriliyam oxaid ek akaarbanik yaugik hai.  ...    Hindi
1  San.1870 ralli Bohra purohitara viruddha maadi...  ...  Kannada
2  Dnyaneshwaranchya kalat mhanje 12vya shatakat ...  ...  Marathi
3  Giniyiligalu tamma panjaragalalli galeeju maad...  ...  Kannada
4  Udaaharanege George Lucas avara star wars na t...  ...  Kannada

[5 rows x 3 columns]


In [ ]:
#df = pd.read_csv(train_path)
#df.columns

Index(['input', 'target', 'lang'], dtype='object')

In [ ]:
#df.head()

In [ ]:
class CommentDataset(Dataset):
    def __init__(self, tokenizer, data_dir, type_path, max_len=30):
        self.path = os.path.join(data_dir, type_path + '.csv')

        self.english = 'target'
        self.hindi = 'input'
        self.lang = "lang"
        self.data = pd.read_csv(self.path)

        self.max_len = max_len
        self.tokenizer = tokenizer
        self.inputs = []
        self.targets = []

        self._build()

    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, index):
        source_ids = self.inputs[index]["input_ids"].squeeze()
        target_ids = self.targets[index]["input_ids"].squeeze()

        src_mask = self.inputs[index]["attention_mask"].squeeze()  # might need to squeeze
        target_mask = self.targets[index]["attention_mask"].squeeze()  # might need to squeeze

        return {"source_ids": source_ids, "source_mask": src_mask, "target_ids": target_ids, "target_mask": target_mask}

    def _build(self):
        for idx in range(len(self.data)):
            input_text,output_text,langu = self.data.loc[idx, self.hindi],self.data.loc[idx, self.english],self.data.loc[idx, self.lang]
   
            input_ = "Romanization:%s langauge: %s" % (output_text,langu)
            target = "%s " %(input_text)

            # tokenize inputs
            tokenized_inputs = self.tokenizer.batch_encode_plus(
                [input_], max_length=80, pad_to_max_length=True, return_tensors="pt",truncation=True
            )
            # tokenize targets
            tokenized_targets = self.tokenizer.batch_encode_plus(
                [target], max_length=80, pad_to_max_length=True, return_tensors="pt",truncation=True
            )

            self.inputs.append(tokenized_inputs)
            self.targets.append(tokenized_targets)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('google/mt5-large')   #,{'use_fast': False})

Downloading:   0%|          | 0.00/376 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/642 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.11M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

In [ ]:
dataset = CommentDataset(tokenizer, '/root/.kaggle/input/hindi-to-hinglish', 'valid1', 30)
print("Val dataset: ",len(dataset))

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2217: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


Val dataset:  624


In [ ]:
data = dataset[20]
print(tokenizer.decode(data['source_ids']))
print(tokenizer.decode(data['target_ids']))

Romanization:क्लब के सदस्य अपने समुदाय के प्रतिष्ठित और सफल लोग थे और सम्मानित परिवार वाले थे उन्होंने बोहेमियन शब्द को अपनी तरह से पुनः परिभाषित किया और उन langauge: Hindi</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>
club ke sadasy apne Samudaay ke pratishthit aur safal Log the aur sammanit Pariwar wale the unhone Bohemian shabd ko apni tarah se pun: Paribhashit kiya aur un </s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>


In [ ]:
#!mkdir result
!mkdir /root/.kaggle/working
!mkdir /root/.kaggle/working/result
!ls /root/.kaggle/input
!pwd

hindi-to-hinglish
/content


In [ ]:
args_dict.update({'data_dir': '/root/.kaggle/input/hindi-to-hinglish', 'output_dir': '/root/.kaggle/working/result', 'num_train_epochs':5,'max_seq_length':30})
args = argparse.Namespace(**args_dict)
print(args_dict)

{'data_dir': '/root/.kaggle/input/hindi-to-hinglish', 'output_dir': '/root/.kaggle/working/result', 'model_name_or_path': 'google/mt5-base', 'tokenizer_name_or_path': 'google/mt5-base', 'max_seq_length': 30, 'learning_rate': 0.0003, 'weight_decay': 0.0, 'adam_epsilon': 1e-08, 'warmup_steps': 0, 'train_batch_size': 2, 'eval_batch_size': 4, 'num_train_epochs': 5, 'gradient_accumulation_steps': 8, 'n_gpu': 1, 'early_stop_callback': False, 'fp_16': False, 'opt_level': 'O1', 'max_grad_norm': 1.0, 'seed': 42}


In [ ]:
checkpoint_callback = pl.callbacks.ModelCheckpoint(
    
    period =1,filepath=args.output_dir, prefix="checkpoint", monitor="val_loss", mode="min", save_top_k=1
)

train_params = dict(
    accumulate_grad_batches=args.gradient_accumulation_steps,
    gpus=args.n_gpu,
    max_epochs=args.num_train_epochs,
    early_stop_callback=False,
    precision= 16 if args.fp_16 else 32,
    amp_level=args.opt_level,
    gradient_clip_val=args.max_grad_norm,
    checkpoint_callback=checkpoint_callback,
    callbacks=[LoggingCallback()],
)

In [ ]:
def get_dataset(tokenizer, type_path, args):
    return CommentDataset(tokenizer=tokenizer, data_dir=args.data_dir, type_path=type_path,  max_len=args.max_seq_length)

In [ ]:
# memory footprint support libraries/code
"""
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
!pip install psutil
!pip install humanize
import psutil
import humanize
import os
import GPUtil as GPU
GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn’t guaranteed
gpu = GPUs[0]
def printm():
 process = psutil.Process(os.getpid())
 print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
 print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
printm()
"""

In [ ]:
!pip install GPUtil

from GPUtil import showUtilization as gpu_usage
from numba import cuda

gpu_usage()
torch.cuda.empty_cache()

import gc
gc.collect()
!nvidia-smi
"""
def free_gpu_cache():
    print("Initial GPU Usage")
    gpu_usage()                             

    torch.cuda.empty_cache()

    cuda.select_device(0)
    cuda.close()
    cuda.select_device(0)

    print("GPU Usage after emptying the cache")
    gpu_usage()

free_gpu_cache()  """ 

In [ ]:
print ("Initialize model")
model = T5FineTuner(args)

trainer = pl.Trainer(**train_params)

Initialize model


Downloading:   0%|          | 0.00/702 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.17G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/376 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.11M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
CUDA_VISIBLE_DEVICES: [0]


In [ ]:
#torch.cuda.memory_summary(device=None, abbreviated=False)

print (" Training model")

trainer.fit(model)

print ("training finished")

print ("Saving model")
model.model.save_pretrained(".kaggle/working/result")

print ("Saved model")

 Training model


NameError: ignored

In [ ]:
!cp -r /root/.kaggle/working/result/pytorch_model.bin /root/.kaggle/working/
!cp -r /root/.kaggle/working/result/config.json /root/.kaggle/working/

In [ ]:
!rm -rf result